In [1]:
import numpy as np
from sklearn.metrics import log_loss

Generate example labels and corresponding predictive probability distributions

*p* is size *(n, n_class) = (10, 3)*

*y* is size *(n,) = (10,)*

In [2]:
p = np.random.rand(10, 3).T
p = (p / p.sum(axis=0)).T
y = np.random.randint(0, 3, (10,))
print(p)
print(y)

[[ 0.14823872  0.6288479   0.22291338]
 [ 0.14333071  0.44374829  0.41292101]
 [ 0.44302287  0.33050584  0.2264713 ]
 [ 0.40842086  0.19952646  0.39205267]
 [ 0.33005722  0.30432011  0.36562267]
 [ 0.25865383  0.38184393  0.35950225]
 [ 0.16319437  0.18920893  0.64759669]
 [ 0.23055204  0.16930447  0.60014349]
 [ 0.36625765  0.32996039  0.30378195]
 [ 0.49521855  0.13573083  0.36905061]]
[0 1 1 0 2 2 1 2 1 0]


In [3]:
log_loss(y, p)

1.0740234117356233

In [4]:
def my_log_loss(y, p):
    classes = np.unique(y)
    y_one_hot = (y[:, np.newaxis] == classes).astype(int)
    print(y_one_hot)
    print(y_one_hot.shape)
    loss = -(y_one_hot * np.log(p)).sum(axis=1)
    return np.mean(loss)

In [5]:
my_log_loss(y, p)

[[1 0 0]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [0 1 0]
 [0 0 1]
 [0 1 0]
 [1 0 0]]
(10, 3)


1.0740234117356233

In [6]:
def my_other_log_loss(y, p):
    p_want = p[np.arange(y.shape[0]), y]
    print(p_want)
    return -np.mean(np.log(p_want))

In [7]:
my_other_log_loss(y, p)

[ 0.14823872  0.44374829  0.33050584  0.40842086  0.36562267  0.35950225
  0.18920893  0.60014349  0.32996039  0.49521855]


1.0740234117356233

In [16]:
import tensorflow as tf
tf_float_type = tf.float64
tf_int_type = tf.int64

def tensorflow_log_loss(y, p):
    classes = np.unique(y)
    class_indices = np.arange(classes.shape[0])
    tf_y = tf.constant(y[:, np.newaxis])
    n = tf.constant(y.shape[0])
    p_pred = tf.constant(p)
    y_one_hot = tf.equal(tf_y, classes)
    y_indices = tf.cast(tf.reduce_sum(tf.where(y_one_hot,
                        tf.ones(tf.shape(y_one_hot)) * class_indices,
                        tf.zeros(tf.shape(y_one_hot))), axis=1),
                        tf_int_type)
    
    indices = tf.cast(tf.range(n), tf_int_type)
    p_want = tf.gather_nd(p_pred, tf.stack([indices, y_indices], axis=1))

    # The cross entropy loss over the training data
    cross_entropy_loss = - tf.reduce_mean(tf.log(p_want))
    sess = tf.Session()
    return sess.run(cross_entropy_loss)

In [17]:
tensorflow_log_loss(y, p)

1.0740234117356233